# <font color=darkred> IDSS32102</font>

<h1><center> Sparse Implementation of LSH</center></h1>

Locality Sensitive Hashing (LSH) can be implemented for both sparse and dense vectors. In this notebook we will implement the algorithm for searching sparse vectors. We will be using k-shingling and minhashing to create our sparse vectors before applying them for search with LSH.



# k-shingling:

We start by defining a few sentences.

In [2]:
a = "flying bird found in the space"
b = "he will not allow you to bring your sticks of dynamite and pet armadillo along"
c = "he figured a few sticks of dynamite were easier than a fishing pole to catch an armadillo"

The first thing we do is create our shingles, we will use k-shingles where k == 2. For longer text it is recommended to create shingles where it is unlikely to produce matching shingles between non-matching text, k values of 7 to 11 would likely produce this outcome.


In [3]:
k = 2
for i in range(len(a) - k+1):
    print(a[i: i+k], end='|')


fl|ly|yi|in|ng|g | b|bi|ir|rd|d | f|fo|ou|un|nd|d | i|in|n | t|th|he|e | s|sp|pa|ac|ce|

In [5]:
k2 = 3
for i in range(len(a)-k+1):
    print (a[i:i+k], end = '/')

fl/ly/yi/in/ng/g / b/bi/ir/rd/d / f/fo/ou/un/nd/d / i/in/n / t/th/he/e / s/sp/pa/ac/ce/

These are our shingles, however, we must remove duplicate values as we are producing a set. We do this using the Python type set. 

1) Define a shingle function to apply shingling to each of our sentences:


In [5]:
def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

In [6]:
a = shingle(a, k)
b = shingle(b, k)
c = shingle(c, k)
print(a)
print(b)
print(c)

{'bi', 'n ', ' f', 'rd', 'nd', 'pa', 'un', 'ce', 'ir', 'ng', 'fl', 'ly', 'sp', 'yi', 'g ', 'ou', 'd ', ' t', ' i', 'in', ' b', 'e ', 'ac', 'he', ' s', 'th', 'fo'}
{'f ', 'lo', 'w ', 'u ', 'l ', 'mi', 'st', ' o', 'ic', 'o ', 'nd', 'am', ' n', 't ', ' y', 'ks', 'an', 'pe', 'il', 'na', 'ma', 'ri', 'br', ' p', 'ng', 'ck', 'ti', ' w', ' d', 'it', 'ad', 'g ', 'yn', 'di', 'ou', ' a', 'd ', ' t', 's ', 'te', 'dy', 'in', 'ot', 'no', 'll', 'ar', ' b', 'wi', 'ow', 'r ', 'al', 'e ', 'rm', 'on', 'ur', 'he', 'yo', 'to', 'et', 'of', ' s'}
{'f ', 'as', 'lo', 'hi', 'w ', ' e', 'n ', 'ha', ' f', 'gu', 'st', ' o', 'mi', 'ic', 'o ', 'er', 'am', 'po', 'si', 'ks', 'fi', 'a ', 'an', 'il', 'na', 're', 'le', 'ea', 'ew', 'ma', ' p', 'fe', 'ng', 'ol', 'ck', 'ti', ' d', 'it', ' w', 'we', 'ad', 'ie', 'g ', 'h ', 'yn', 'di', ' a', 'd ', 'sh', ' t', 's ', 'te', 'dy', 'in', 'tc', 'll', 'ar', 'is', 'at', 'ig', 'ch', 'r ', ' c', 'ca', 'e ', 'rm', 'ur', 'he', 'to', 'of', ' s', 'th', 'ed'}


Now that we have our three shingles we create a shingle vocabulary by create a union between all three sets.

In [7]:
vocab = list(a.union(b).union(c))
print(vocab)

['hi', 'w ', 'bi', 'n ', 'ha', 'mi', 'er', 'po', 'pa', ' n', 'fi', 'il', 'na', 're', 'ri', 'un', 'ck', 'ti', ' d', 'it', 'ad', 'we', 'sp', 'ou', ' a', 'd ', 'sh', ' t', ' i', 'dy', 'in', 'ot', 'no', 'll', ' b', 'at', 'wi', 'ow', 'r ', 'e ', 'on', 'ur', 'he', 'of', 'th', 'ed', 'f ', 'as', 'lo', ' e', 'u ', 'l ', ' f', 'gu', 'st', ' o', 'ic', 'o ', 'rd', 'nd', 'am', 'si', 't ', ' y', 'ks', 'an', 'a ', 'pe', 'ma', 'le', 'ea', 'ew', 'ce', 'br', 'ir', ' p', 'fe', 'ng', 'ol', ' w', 'fl', 'ly', 'yi', 'ie', 'g ', 'h ', 'yn', 'di', 's ', 'te', 'tc', 'ar', 'is', 'ig', 'ch', ' c', 'ca', 'al', 'ac', 'rm', 'et', 'yo', 'to', ' s', 'fo']


Using this vocab we can create one-hot encoded sparse vectors to represent our shingles.

In [8]:
a_1hot = [1 if x in a else 0 for x in vocab]
b_1hot = [1 if x in b else 0 for x in vocab]
c_1hot = [1 if x in c else 0 for x in vocab]
print(a_1hot)

[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]


So we now have one-hot encoded sparse vectors we can move onto minhashing.

# Minhashing

Minhashing is the next step in our process. After creating our shingle sets we use minhashing to create signatures from those sets.

To create our minhashing function we build several hash functions, each will randomly count from 1 to len(vocab) + 1 - creating a random vector:


In [9]:
hash_ex = list(range(1, len(vocab)+1))
print(hash_ex)  # we haven't shuffled yet

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105]


In [26]:
from random import shuffle

shuffle(hash_ex)
print(hash_ex)

[52, 6, 95, 25, 70, 98, 18, 39, 80, 54, 64, 43, 79, 40, 28, 56, 65, 16, 82, 53, 13, 51, 93, 8, 87, 3, 10, 1, 86, 104, 90, 26, 105, 27, 68, 49, 24, 92, 59, 78, 5, 76, 19, 88, 67, 71, 62, 73, 72, 57, 60, 75, 4, 30, 66, 74, 31, 100, 45, 17, 14, 94, 15, 96, 29, 21, 12, 101, 102, 63, 37, 50, 99, 9, 35, 11, 97, 42, 32, 91, 23, 55, 89, 77, 83, 38, 36, 33, 34, 7, 81, 69, 46, 103, 22, 20, 47, 58, 2, 85, 48, 61, 41, 84, 44]


We now have a randomized list of integers which we can use in creating our hashed signatures. What we do now is begin counting from 1 through to len(vocab) + 1, extracting the position of this number in our new hash_ex list, like so (for the first few integers):


In [35]:
for i in range(1, 5):
    print(f"{i} -> {hash_ex.index(i)}")

1 -> 27
2 -> 98
3 -> 25
4 -> 52



What we do with this is count up from 1 to len(vocab) + 1 and find if the resultant hash_ex.index(i) position in our one-hot encoded vectors contains a positive value (1) in that position, like so:


In [36]:
for i in range(1, len(vocab)+1):
    idx = hash_ex.index(i)
    signature_val = a_1hot[idx]
    print(f"{i} -> {idx} -> {signature_val}")
    if signature_val == 1:
        print('match!')
        break

1 -> 27 -> 1
match!


That gives us a first signature of some value. But this is just a single value, and it takes many values to create a signature (100 for example).

So, how to we generate these other values? By using more hash functions! Let's generate a set of hash functions to create a signature vector of length 20.

hash_funcs = []

for _ in range(20)


In [13]:
hash_funcs = []

for _ in range(20):
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    hash_funcs.append(hash_ex)

for i in range(3):
    print(f"hash function {i+1}:")
    print(hash_funcs[i])

hash function 1:
[62, 12, 21, 26, 42, 90, 96, 85, 38, 101, 89, 9, 36, 84, 17, 82, 32, 6, 68, 104, 99, 94, 23, 71, 18, 81, 61, 103, 4, 73, 93, 86, 47, 22, 25, 13, 48, 39, 31, 95, 5, 83, 57, 92, 52, 16, 8, 19, 87, 10, 88, 105, 44, 65, 97, 49, 66, 46, 50, 51, 64, 30, 34, 37, 53, 33, 43, 91, 58, 1, 55, 2, 56, 100, 41, 54, 78, 40, 7, 60, 70, 74, 98, 76, 45, 77, 69, 15, 28, 11, 102, 29, 72, 24, 59, 3, 79, 63, 27, 20, 67, 80, 35, 75, 14]
hash function 2:
[80, 51, 25, 41, 4, 21, 77, 96, 105, 50, 100, 36, 23, 64, 42, 9, 66, 95, 12, 48, 44, 83, 88, 61, 86, 34, 99, 56, 1, 98, 76, 40, 11, 35, 46, 45, 72, 31, 28, 92, 63, 19, 6, 53, 103, 101, 29, 38, 22, 82, 85, 74, 70, 2, 93, 54, 59, 75, 71, 94, 10, 102, 58, 20, 39, 30, 33, 37, 5, 55, 47, 69, 7, 62, 17, 49, 15, 73, 97, 79, 91, 24, 67, 16, 68, 26, 43, 90, 81, 18, 89, 13, 52, 78, 57, 65, 87, 84, 3, 27, 14, 8, 60, 104, 32]
hash function 3:
[60, 100, 22, 45, 27, 93, 76, 94, 47, 79, 32, 97, 64, 9, 44, 68, 72, 3, 98, 4, 20, 92, 87, 16, 48, 18, 73, 36, 26

We're only showing the first three hash functions here - we have 20 in total. To create our signatures we simply process each one-hot vector through each hash function, appending the output value to our signature for that vector.

2) Complete the following signature code:


In [14]:
signature = []

for func in hash_funcs:
    for i in range(1, len(vocab)+1):
        idx = func.index(i)
        signature_val = a_1hot[idx]
        # ...

#print(signature)

And there we have our minhash produced signature for a. Let's clean up the code and formalize the process a little.


In [15]:

def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, size+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 20)

3) Write a signature function:

In [16]:
def create_hash(vector: list):
    # use this function for creating our signatures (eg the matching)
    return signature

In [17]:
a_sig = create_hash(a_1hot)
b_sig = create_hash(b_1hot)
c_sig = create_hash(c_1hot)

In [18]:
#print(a_sig)
#print(b_sig)

We now have our three minhashed signatures! These signatures, despite being seemingly randomized, will on average have the very similar Jaccard similarity values as our previous sparse vectors. We have reduced the dimensionality of our vectors significantly - but maintained the same information!

4) Write a Jaccard similarity function:


In [19]:
def jaccard(a: set, b: set):
    return #...


a should have lower similarity with b and c:

In [20]:
jaccard(a, b), jaccard(set(a_sig), set(b_sig))

(None, None)

In [21]:
jaccard(a, c), jaccard(set(a_sig), set(c_sig))

(None, None)

In [22]:
jaccard(b, c), jaccard(set(b_sig), set(c_sig))

(None, None)

We're now ready to move onto the LSH process.

## Locality Sensitive Hashing

The approach we will be taking in this notebook is break our signature vector into multiple bands, creating several sub-vectors.

We then hash each of these sub-vectors into a set of buckets, if we find that two sub-vectors from two signature vectors collide (end up in the same hash bucket) we take the two full signature vectors as candidate pairs - which we then compare in full with a similarity metric (like Jaccard similarity, cosine similarity, etc).

There is no 'set' way to hash our signature vectors, and in-fact the simplest approach is to check for equivalence across sub-vectors, which will be our approach.

First, we must define the number of buckets b we would like to create. It's important to note that each bucket must contain an equal number of rows r - and so our signature length must be divisible by b.


In [23]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

We'll start by splitting into 10 bands, creating rows of 2 - on the small side to be used in a genuine LSH function but good for our example (we'll explore different r and b values soon).

Let's start with our b and c vectors, which should hopefully match in at least one band.

In [24]:
band_b = split_vector(b_sig, 10)
band_b

ValueError: range() arg 3 must not be zero

In [ ]:
band_c = split_vector(c_sig, 10)
band_c

Check if they match (we'll rewrite some of this into Numpy soon).

In [ ]:
for b_rows, c_rows in zip(band_b, band_c):
    if b_rows == c_rows:
        print(f"Candidate pair: {b_rows} == {c_rows}")
        # we only need one band to match
        break

And let's do the same for a.

In [ ]:
band_a = split_vector(a_sig, 10)

In [ ]:
for a_rows, b_rows in zip(band_a, band_b):
    if a_rows == b_rows:
        print(f"Candidate pair: {a_rows} == {b_rows}")
        # we only need one band to match
        break

In [ ]:
for a_rows, c_rows in zip(band_a, band_c):
    if a_rows == c_rows:
        print(f"Candidate pair: {a_rows} == {c_rows}")
        # we only need one band to match
        break

Okay great, so even with this very simple implementation - we most likely managed to identify sentences b and c as candidate pairs, and identify a as a non-candidate.


## Tuning LSH

We can visualize the probability of returning a candidate pair vs the similarity of the pair for different values of r and b (rows and bands respectively) like so:


In [ ]:
def probability(s, r, b):
    # s: similarity
    # r: rows (per band)
    # b: number of bands
    return 1 - (1 - s**r)**b

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
results = pd.DataFrame({
    's': [],
    'P': [],
    'r,b': []
})

for s in np.arange(0.01, 1, 0.01):
    total = 100
    for b in [100, 50, 25, 20, 10, 5, 4, 2, 1]:
        r = int(total/b)
        P = probability(s, r, b)
        results = results.append({
            's': s,
            'P': P,
            'r,b': f"{r},{b}"
        }, ignore_index=True)

In [ ]:
sns.lineplot(data=results, x='s', y='P', hue='r,b')